In [ ]:
import numpy as np
import pandas
import math

If possible, update your sklearn version to 1.3.2 to reduce variance in the versions.

In [ ]:
#!pip3 install scikit-learn==1.3.2

In [ ]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

## Gradient Descent

- Functions from description

In [ ]:
def sigma(x):
    return math.exp(x)/(math.exp(x) + 1) 

def f(u,v,b):
    return -math.log(sigma(u+b)) - math.log(sigma(v+b)) - math.log(sigma(-u/2 - v/2 - b)) + (u**2 + v**2 + b**2)/100

- Gradient of f is a vector (list in python) consisting of three partial derivatives. Each partial derivative is on one of coordinate of the point (u,v,b).

In [ ]:
#Write only the derivatives of the terms (which is very convenient using sigma) and let the code compute the result of the partial derivative
def alt_grad_f(u,v,b):
    a_dfu = -sigma(-(u+b)) - 0 + sigma((u+v+2*b)/2)/2 + u/50

    a_dfv = -sigma(-(v+b)) - 0 + sigma((u+v+2*b)/2)/2 + v/50

    a_dfb = -sigma(-(u+b)) - sigma(-(v+b)) + sigma((u+v+2*b)/2) + b/50

    return [a_dfu, a_dfv, a_dfb]

Gradient descent function

In [ ]:
def gradient_descent(f, grad_f, eta, u_0, v_0, b_0, max_iter=100):
    curX = [u_0, v_0, b_0]  #Current point coordinates
    best = 1000000000 #Initial best score
    min_t = 1 #Initial best step
    t = 0
    while t < max_iter: 
        t = t + 1
        step_size = eta(t) #Get step size at each iteration
        #print(step_size,t)
        #Calculate the new points
        gradient_result  = grad_f(curX[0],curX[1],curX[2]) 
        for i in range (3):
            curX[i]= curX[i] - step_size * gradient_result[i]
        
        #Check whether f(u_t,v_t,b_t) is the smaller than the current smallest result
        cur_f = f(curX[0],curX[1],curX[2])
        #print(cur_f)
        if(cur_f < best): 
            best = cur_f
            min_t = t

    curX.append(best)
    curX.append(min_t) #return u_100, v_100, b_100, smallest result and its t
    return curX 

#Step-size functions
def eta_const(t, c=0.2):
    return c

def eta_sqrt(t,c=0.5):
    return c / math.sqrt(t+1)

def eta_multistep(t, milestones, c, eta_init):
    i = 0
    while i < len(milestones):
        if(t < milestones[i]):
            return eta_init * (c**i)
        i+=1
    return eta_init * (c**i)

#Initial points
u_0 = 4
v_0 = 2
b_0 = 1

#Print result
def print_result(result):
    #print(result)

    u_100 = result[0]
    v_100 = result[1]
    b_100 = result[2]

    answer = f(u_100, v_100, b_100)
    print("f(u_100, v_100, b_100) = ", round(answer,3))
    print("Min val", round(result[3],3))

Results

In [ ]:
#Part A
result = gradient_descent(f, alt_grad_f, lambda t: eta_const(t, c=0.2) ,u_0, v_0, b_0, max_iter=100)
print_result(result)

In [ ]:
#Part B
result = gradient_descent(f, alt_grad_f, lambda t: eta_sqrt(t, c=0.5) ,u_0, v_0, b_0, max_iter=100)
print_result(result)

In [ ]:
#Print C
result = gradient_descent(f, alt_grad_f, lambda t: eta_multistep(t, milestones=[30,50,80], c=0.5, eta_init=1.0) ,u_0, v_0, b_0, max_iter=100)
print_result(result)

## Coordinate Descent

In [ ]:
def f(x):
    return math.exp(x[0]-x[1]+1) + math.exp(x[1]-x[2]+2) + math.exp(x[2]-x[0]+3)

#Partial derivative = 0 for x1
def argmin_x1(x):
    return  (x[2] + x[1] + 2)/2   #(x3 + x2 + 2) / 2

#Partial derivative = 0 for x2
def argmin_x2(x):
    return (x[2] + x[0] - 1)/2    #(x3 + x1 - 1) / 2

#Partial derivative = 0 for x3
def argmin_x3(x):
    return (x[0] + x[1] - 1)/2    #(x1 + x2 - 1) / 2

def argmin_result(x):
    return [argmin_x1(x),argmin_x2(x),argmin_x3(x)]

x0 = [2,3,4]
print(argmin_result(x0))

In [ ]:
def coordinate_descent(f, argmin, x_0, max_iter=100):
    x_t = x_0
    t = 0
    while t < max_iter:
        t+=1
        for i in range (3):
            x_t[i] = argmin[i](x_t)
            print(x_t[i],t) #print coordinates and iteration

        #Print result of iteration
        cur_f = f(x_t)
        print(cur_f)
    return x_t

x_0 = [1,20,5]
argmin = [argmin_x1,argmin_x2,argmin_x3] 
coordinate_descent(f, argmin, x_0, max_iter=100)


## Regression - Polynomial features

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
california = fetch_california_housing()
print(california.DESCR)

Creating the data matrix

In [ ]:
D = california.data
y = california.target
n,d = D.shape
print(n,d)

In [ ]:
scaler = StandardScaler()
D_scaled = scaler.fit_transform(D)
print(D)

Creating a design matrix with polynomial features

In [ ]:
poly = PolynomialFeatures(2,include_bias=True)
X = poly.fit_transform(D_scaled)
poly.get_feature_names_out(california.feature_names)
print("Original feature shape:", D.shape)
print("Design matrix shape with polynomial features (degree 2):", X.shape)

In [ ]:
for i in  range(len(poly.get_feature_names_out(california.feature_names))):
    if poly.get_feature_names_out(california.feature_names)[i] == 'MedInc':
        print('MedInc: ' + str(i))
    if poly.get_feature_names_out(california.feature_names)[i] == 'MedInc AveBedrms':
        print('MedInc AveBedrms: ' + str(i))
    if poly.get_feature_names_out(california.feature_names)[i] == 'HouseAge AveBedrms':
        print('HouseAge AveBedrms:' + str(i))

In [ ]:
XT_X = X.T @ X
beta = (np.linalg.inv(XT_X) @ X.T) @ y

In [ ]:
beta[1], beta[12], beta[19]

In [ ]:
lambda_ = 0.1
I = np.eye(X.shape[1])
XT_y = X.T @ y

ridge_beta = np.linalg.inv(XT_X + lambda_ * n * I) @ XT_y
ridge_beta

In [ ]:
round(ridge_beta[1],5), round(ridge_beta[12],5), round(ridge_beta[19],5)

## Bias-var Trade Off

In [ ]:
#True regression function
# pls appear
def f_star(x):
    return math.tan(math.pi*x)

#Regression models
def f_d1(x):
    return x+0.2

def f_d2(x):
    return 3*x+0.3

def f_d3(x):
    return 5*x+0.1

#Calculate expected result from the sample models with x0 = 0 and use it to get the bias^2
e_fd = (f_d1(0) + f_d2(0) + f_d3(0)) / 3 #using the hint
bias_2 = round((f_star(0) - e_fd) ** 2,4)
#print("E = ", e_fd)
print("Answers")
print("bias^2 = ", bias_2)

#Using the formula from the slides (just the standard way of computing variance)
ee_fd = ((e_fd - f_d1(0))**2 + (e_fd - f_d2(0))**2 + (e_fd - f_d3(0)))/3
variance = round(ee_fd **2,5)
#print(e_fd - f_d3(0))
#print("EE = ", ee_fd)
print("variance = ", variance)

## Naive Bayes
From the 20Newsgroups dataset we fetch the documents belonging to three categories, which we use as classes.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
categories = ['rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey']
train = fetch_20newsgroups(subset='train', categories=categories)
test = fetch_20newsgroups(subset='test', categories=categories)

For example, the first document in the training data is the following one:

In [ ]:
print(train.data[0])

The classes are indicated categorically with indices from zero to two by the target vector. The target names tell us which index belongs to which class.

In [ ]:
y_train = train.target
y_train

In [ ]:
train.target_names

We represent the documents in a bag of word format. That is, we create a data matrix ``D`` such that ``D[j,i]=1`` if the j-th document contains the i-th feature (word), and ``D[j,i]=0`` otherwise. 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words="english", min_df=5,token_pattern="[^\W\d_]+", binary=True)
D = vectorizer.fit_transform(train.data)
D_test = vectorizer.transform(test.data)

We get the allocation of feature indices to words by the following array, containing the vocabulary.

In [ ]:
vectorizer.get_feature_names_out()

In [ ]:
num_features = len(vectorizer.get_feature_names_out())

For example, the word `naive` has the index 4044.

In [ ]:
naive_id = np.where(vectorizer.get_feature_names_out() == 'naive')[0]
naive_id

In [ ]:
y0 = len([a for a in y_train if a == 0])/len(y_train)
y1 = len([a for a in y_train if a == 1])/len(y_train)
y2 = len([a for a in y_train if a == 2])/len(y_train)
y3 = len([a for a in y_train if a == 3])/len(y_train)

y0, y1, y2, y3

In [ ]:
alpha = 1e-5  # Smoothing parameter
vocabulary = vectorizer.get_feature_names_out()

V = len(vocabulary)  # Vocabulary size
unique_classes = np.unique(y_train)

log_probabilities = {}

for k in unique_classes:
    # Get indices of documents in class k
    class_indices = np.where(y_train == k)[0]
    
    # Number of documents in class k
    N_k = len(class_indices)
    
    # Number of documents in class k containing the word "naive"
    N_naive_k = D[class_indices, naive_id].sum()
    
    # Compute smoothed probability
    prob = (N_naive_k + alpha) / (N_k + alpha * V)
    log_probabilities[k] = np.log(prob)

# Print log-probabilities
for k, log_prob in log_probabilities.items():
    print(f"log p(x_naive = 1 | y = {k}) = {log_prob:.6f}")

In [ ]:
# Assuming the following values are known or calculated: 5C
# 1. Priors (p(y=k))
class_priors = [0.25, 0.25, 0.25, 0.25]  # Replace with actual priors from 5a

# 2. Likelihoods (p(x_word=1 | y=k)) from 5b
likelihoods = {
    'autos': [0.001, 0.0001, 0.0001, 0.0001],
    'motorcycles': [0.0001, 0.002, 0.0001, 0.0001],
    'baseball': [0.0001, 0.0001, 0.003, 0.0001],
    'hockey': [0.0001, 0.0001, 0.0001, 0.004],
}

# Function to compute posterior probabilities using Bayes' theorem
def compute_posterior(word, class_priors, likelihoods):
    # Extract likelihoods for the given word
    word_likelihoods = likelihoods[word]
    
    # Compute the denominator p(x_word=1)
    p_x_word = sum(word_likelihoods[k] * class_priors[k] for k in range(len(class_priors)))
    
    # Compute posterior probabilities for each class
    posteriors = [(word_likelihoods[k] * class_priors[k]) / p_x_word for k in range(len(class_priors))]
    return posteriors

# Example: Compute for each word
words = ['autos', 'motorcycles', 'baseball', 'hockey']
for word in words:
    posteriors = compute_posterior(word, class_priors, likelihoods)
    for k, posterior in enumerate(posteriors):
        print(f"p(y = {k} | x_{word} = 1) = {posterior:.6f}")


## Decision Tree

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
D, y = iris.data, iris.target

In [ ]:
print(iris.DESCR)

In [ ]:
def gini_imp(y):
    temp = 0
    for i in range(3):
        temp += (len([a for a in y if a == i])/len(y))**2
    g_impurity = 1 - temp
    return g_impurity

In [ ]:
gini_imp(y)

In [ ]:
D[0]

In [ ]:
small_sep_id = []
big_sep_id = []
for i in range(150):
    if(D[i][0] <= 5.84):
        small_sep_id.append(i)
    else:
        big_sep_id.append(i)
#print(small_sep_id)
#print(big_sep_id)
y_small = [y[j] for j in small_sep_id]
y_big = [y[j] for j in big_sep_id]
#print(gini_imp(y_small))
#print(gini_imp(y_big))
cost = (len(small_sep_id)/150 * gini_imp(y_small) + len(big_sep_id)/150 * gini_imp(y_big)) - gini_imp(y)
print(cost)

## SVM

In [ ]:
# Standard scientific Python imports
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
digits = datasets.load_digits()

_, axes = plt.subplots(nrows=1, ncols=4, figsize=(10, 3))
for ax, image, label in zip(axes, digits.images, digits.target):
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation="nearest")
    ax.set_title("Label %i" % label)

In [ ]:
# flatten the images
n = len(digits.images)
D = digits.images.reshape((n, -1))
y = digits.target

# Split data into 70% train and 30% test subsets
D_train, D_test, y_train, y_test = train_test_split(
    D, y, test_size=0.3, shuffle=False
)

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel='rbf', gamma=0.0008, C=0.9)
model = svc.fit(D_train, y_train)
print(svc.score(D_test, y_test))

In [ ]:
# Calculate how many supporting vectors classes 0 and 1 have in total
support_vectors = svc.n_support_
all_support_vectors_0_1 = support_vectors[0] + support_vectors[1]
all_support_vectors_0_1

In [ ]:
# Compute how many supporting vectors are distinguished between classes 0 and 1
all_coeficients_0_1 = model.dual_coef_[0, :all_support_vectors_0_1]
distinguished_coeficients_0_1 = all_coeficients_0_1[np.where(all_coeficients_0_1 != 0)]
distinguished_coeficients_0_1.size

In [ ]:
# Extract support vectors and dual coefficients from training
support_vectors = svc.support_vectors_
dual_coef = svc.dual_coef_

# Separate the support vectors for class 0 and class 1
# Dual coefficients have two rows for binary classification
class_0_indices = (dual_coef[0] > 0).nonzero()[0]
class_1_indices = (dual_coef[0] < 0).nonzero()[0]

# Get the top 4 most influential support vectors for each class
top_4_class_0 = support_vectors[class_0_indices[:4]]
top_4_class_1 = support_vectors[class_1_indices[:4]]    


# Plot the support vectors
fig, axes = plt.subplots(2, 4, figsize=(10, 5))
for i, sv in enumerate(top_4_class_0):
    axes[0, i].imshow(sv.reshape(8, 8), cmap='gray')
    axes[0, i].set_title(f"Class 0, SV {i+1}")
    axes[0, i].axis('off')

for i, sv in enumerate(top_4_class_1):
    axes[1, i].imshow(sv.reshape(8, 8), cmap='gray')
    axes[1, i].set_title(f"Class 1, SV {i+1}")
    axes[1, i].axis('off')

plt.suptitle("Top 4 Support Vectors for Classes 0 and 1")
plt.show()


In [ ]:
# Create an SVC with RBF kernel
svc = SVC()

# Define the parameter grid
param_grid = {
    'kernel':['rbf'],
    'gamma': [0.0001, 0.0006, 0.001, 0.006],
    'C': [0.6, 0.8, 1, 2, 3, 4, 6]
}

# Create the GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, scoring='accuracy', cv=5)

# Train the model on the whole dataset D
grid_search.fit(D, y)

# Get the best parameters and the best accuracy
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Accuracy:", best_accuracy)